In [8]:
import os
import pandas as pd
from tiled.client import from_uri

In [9]:
import pandas as pd

def parse_linescan_log(file_path):
    """
    Parse a linescan log file into a pandas DataFrame.
    
    Each entry in the file should consist of 4 lines:
      1. Timestamp (e.g., "2024-02-12T09-33-05")
      2. Mode (e.g., "mode = xafs_x/It")
      3. UID   (e.g., "uid = e9979416-fd6d-4ea5-9d56-2257f52c9572")
      4. Position (e.g., "position = 144.1771555806452")
    
    The resulting DataFrame uses the UID as the index and includes:
      - timestamp
      - motor (parsed from mode)
      - detector (parsed from mode)
      - position
    """
    data = []
    
    with open(file_path, 'r') as f:
        # Remove blank lines and strip whitespace
        lines = [line.strip() for line in f if line.strip()]
    
    # Process entries in blocks of 4 lines.
    for i in range(0, len(lines), 4):
        try:
            ts_line = lines[i]
            mode_line = lines[i + 1]
            uid_line = lines[i + 2]
            pos_line = lines[i + 3]
        except IndexError:
            # If there are incomplete blocks at the end, skip them.
            break

        # Parse timestamp
        timestamp = ts_line
        
        # Parse mode and split into motor and detector.
        # e.g., "mode = xafs_x/It"
        mode_value = mode_line.split("=", 1)[1].strip()
        motor, detector = mode_value.split("/", 1)
        
        # Parse UID (e.g., "uid = e9979416-fd6d-4ea5-9d56-2257f52c9572")
        uid = uid_line.split("=", 1)[1].strip()
        
        # Parse position (e.g., "position = 144.1771555806452")
        position = float(pos_line.split("=", 1)[1].strip())
        
        data.append({
            "timestamp": timestamp,
            "motor": motor,
            "detector": detector,
            "position": position,
            "uid": uid
        })
    
    df = pd.DataFrame(data)
    df.set_index("uid", inplace=True)
    return df

# Example usage:
# df = parse_linescan_log("path/to/linescan_evaluation.txt.1")
# print(df)


In [14]:
df1 = parse_linescan_log(os.path.join("datasets", "linescan_evaluation.txt.1"))
df2 = parse_linescan_log(os.path.join("datasets", "linescan_evaluation.txt.2"))
df = pd.concat([df1, df2], axis=0)

In [15]:
df

,timestamp,motor,detector,position
uid,,,,
e9979416-fd6d-4ea5-9d56-2257f52c9572,2024-02-12T09-33-05,xafs_x,It,144.177156
2d8b6741-e589-4e6b-8c16-f3ab6bbc336e,2024-02-12T09-36-20,xafs_x,It,144.150454
2e30454f-a5ac-48f9-a3bf-5e1b15e544b5,2024-02-12T09-40-38,xafs_x,It,143.644743
a925e08a-7d1c-4e44-a8d5-126bcb62dc9e,2024-02-12T09-45-30,xafs_x,It,144.097048
76c348a3-5bb4-4b21-93e6-9e25995c6eaf,2024-02-12T09-48-23,xafs_x,It,141.169582
...,...,...,...,...
e653d400-895a-428f-88ca-d83e9b07907f,2025-01-22T16-52-36,xafs_x,Xs,109.448655
d8ae25e2-4096-4143-8c5c-63e18baf7a57,2025-01-23T12-42-23,xafs_y,Xs,171.603844
5ae5fdad-7b26-4901-bf78-cfda948de3a8,2025-01-24T08-09-24,xafs_x,Xs,161.292126


In [16]:
container = from_uri("https://tiled.nsls2.bnl.gov")

In [17]:
container

<Container {'tst', 'amx', 'bmm', 'chx', 'cms', 'csx', 'esm', ...} ~35 entries>

In [18]:
bmm_catalog = container["bmm"]["raw"]

In [19]:
bmm_catalog

<Catalog {}>